In [82]:
import pandas as pd
import re
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [83]:
skinFiles = ['skin/skin_data_'+str(i+1)+'.csv' for i in range(7)]
list_ = []
for file_ in skinFiles:
    df = pd.read_csv(file_, index_col=None, header=0)
    df['subject']='dermatology'
    list_.append(df)

In [84]:
df = pd.read_csv('sex/amipregnant_data_1.csv', index_col=None, header=0)
df['subject']='am-i-pregnant'
list_.append(df)

df = pd.read_csv('sex/birthcontrolmen_data_1.csv', index_col=None, header=0)
df['subject']='birth-control-men'
list_.append(df)

df = pd.read_csv('sex/birthcontrolwomen_data_1.csv', index_col=None, header=0)
df['subject']='birth-control-women'
list_.append(df)

df = pd.read_csv('sex/hivaids_data_1.csv', index_col=None, header=0)
df['subject']='hiv-aids'
list_.append(df)

df = pd.read_csv('sex/ibs_data_1.csv', index_col=None, header=0)
df['subject']='irritable-bowel-syndrome'
list_.append(df)

df = pd.read_csv('sex/menstruationwomen_data_1.csv', index_col=None, header=0)
df['subject']='menstruation'
list_.append(df)

df = pd.read_csv('sex/penismen_data_1.csv', index_col=None, header=0)
df['subject']='birth-control-men'
list_.append(df)

df = pd.read_csv('sex/sexualhealthmen_data_1.csv', index_col=None, header=0)
df['subject']='penis'
list_.append(df)

df = pd.read_csv('sex/sexualhealthwomen_data_1.csv', index_col=None, header=0)
df['subject']='sexual-health-men'
list_.append(df)

df = pd.read_csv('sex/stress_data_1.csv', index_col=None, header=0)
df['subject']='sexual-health-women'
list_.append(df)

df = pd.read_csv('sex/vaginalwomen_data_1.csv', index_col=None, header=0)
df['subject']='vagina'
list_.append(df)

In [85]:
df = pd.read_csv('sex/doctorslounge_data_1.csv', index_col=None, header=0)
list_.append(df)

In [86]:
df=pd.concat(list_,ignore_index=True)
df=df.drop(df.columns[0],1)

In [87]:
diseases = pd.read_csv("disease_list.csv",index_col=None, header=None)
diseases = diseases[1].str.strip().str.replace(' +',' ').str.lower()
d_list = diseases.loc[:373].tolist()

diseases = pd.read_csv("disease_list_cdc.csv",index_col=0, header=0)
diseases = diseases['name'].str.strip().str.replace(' +',' ').str.lower()
d_list=d_list+diseases.tolist()

#d_list=d_list+[words for segments in d_list for words in segments.split()]
pd_dis=pd.DataFrame(d_list).drop_duplicates(0)
d_list = pd_dis[0].tolist()

In [88]:
#txt = df.loc[df['response_type']=='doctor','response'].str.lower().tolist()
#df['response_time']=pd.to_datetime(df['response_time'].str.replace('replied ',''))

In [ ]:
df.shape

(12126, 7)

In [ ]:
ti=0
for post in df['post'].unique():
    if(ti%100==0):
        print ti
    ti=ti+1
    subdf = df['post']==post
    txt = df.loc[subdf & (df['response_type']=='doctor'),'response']
    if(not txt.empty):
        txt2 = df.loc[subdf & (df['response_type']=='asker'),'response']
        ask_time = df.loc[subdf & (df['response_type']=='asker'),'response_time']
        res_time = df.loc[subdf & (df['response_type']=='doctor'),'response_time']
        diag = pd_dis.loc[[re.search(r'\b'+word+r'\b', txt.str.lower().tolist()[0])>0 for word in d_list],0].tolist()
        sdiag= pd_dis.loc[[re.search(r'\b'+word+r'\b', txt2.str.lower().tolist()[0])>0 for word in d_list],0].tolist()
        df.loc[subdf,'diagnosis_count'] = len(diag)
        df.loc[subdf,'selfdiagnosis_count'] = len(sdiag)
        df.loc[subdf,'label']=', '.join(diag)
        for w in diag:
            if 'diag_'+w in df.columns:
                df.loc[subdf,'diag_'+w]=df.loc[subdf,'diag_'+w]+1
            else:
                df['diag_'+w]=0
                df.loc[subdf,'diag_'+w]=1
        for w in sdiag:
            if 'namedis_'+w in df.columns:
                df.loc[subdf,'namedis_'+w]=df.loc[subdf,'namedis_'+w]+1
            else:
                df['namedis_'+w]=0
                df.loc[subdf,'namedis_'+w]=1
        df.loc[subdf & (df['response_type']=='asker'),'lag']=res_time.tolist()[0]
    else:
        df=df.loc[[not i for i in subdf.tolist()],:]
df['lag']=df['lag']-df['response_time']

In [ ]:
df = df.loc[df['response_type']=='asker',:]
df.to_csv("compiled_data_toclassify.csv")

In [ ]:
g = sns.jointplot(x='selfdiagnosis_count', y='diagnosis_count', data=df, kind="hex", color="m")
g.plot_joint(plt.scatter, c="k", s=30, linewidth=1, marker="x")
g.ax_joint.collections[0].set_alpha(0)

In [ ]:
sns.distplot(df['lag'].apply(lambda x: float(x.days)))